## Contents
* [1. Background](#1.-Background)
    * [1.1. Objective](#1.1.-Objective)
    * [1.2. Additional information](#1.2.-Additional-information)
* [2. Import Train Data](#2.-Import-Data)
    * [2.1. Imports](#2.1.-Imports)
    * [2.2. Initial summary of data](#2.2.-Initial-summary-of-data)
* [3. Cleaning](#3.-Cleaning)
    * [3.1. Check for null values](#3.1.-Check-for-null-values)
    * [3.2. Change NaN values](#3.2.-Change-NaN-values)
    * [3.3. Change Y and N values](#3.3.-Change-Y-and-N-values)
    * [3.4. Drop repetitive features](#3.4.-Drop-repetitive-features)
* [4. Export Data](#4.-Export-Data)

---
## 1. Background
---
*  Following a company-wide performance review, ABC Estate Agency noticed a rising trend of misvaluation of Housing Development Board (HDB) resale flats by its estate agents. These cases of misvaluation are a violation of the national Code of Ethics (where guilty estate agents are subjected to disciplinary and legal action) and adversely affect the company's reputation.
* As an effort to reduce manual effort and human errors, and to help its estate agents more accurately and timely synthesise large amounts of HDB-resale-flat-related data, the management team of ABC Estate Agency assigned me a task to create a prediction model of HDB resale flats.

### 1.1. Objective
* Create a prediction model of HDB resale flat prices with at least a $R^{2}$ of 0.8.

### 1.2. Additional information
* The Council for Estate Agencies (CEA), Singapore's regulatory body for the real estate agency industry, provides a Professional Service Manual that all estate agents must comply to. Under section 2.1, when advising any persons on property prices, estate agents must ensure that the data provided on prices are current and accurate, and that the range and amount of data used are comprehensive.
* Free online HDB flat value calculators focus on several common factors: property location, property types (e.g. how many rooms per flat), floor level and floor area. These calculators can be taken as anecdotal evidence that the aforementioned factors are key to the evaluation of the price of a HDB flat.

Sources:
* [CEA - Professional Service Manual](https://www.cea.gov.sg/docs/default-source/legislation-guideliness/Practice-Guidelines-Circulars/professional-service-manual/practice-guidelines---professional-service-manual-(pdf-755kb).pdf)
* [CEAnswers: providing advice on price trends and collecting cheques from prospective buyers](https://www.cea.gov.sg/docs/default-source/module/newsletter/4-2019/ceanswers.html)
* [Money Smart - Property Valuation Calculator](https://www.moneysmart.sg/home-loan/property-valuation-calculator)
* [99.co - Property Value Tool](https://www.99.co/singapore/property-value-tool/form/adrry9rfmUXNcW9KwvtUUg8o)

---
## 2. Import Train Data
---

### 2.1. Imports

In [89]:
import numpy as np
import pandas as pd
import math
import os

### 2.2. Initial summary of data

In [90]:
hdb_df = pd.read_csv('datasets/train.csv')
pd.set_option('display.max_columns', None)  # to see all columns
print(hdb_df.shape)
print(hdb_df.head())
print(hdb_df.info())

C:\Users\Edmon\AppData\Local\Temp\ipykernel_138480\4253937880.py:1: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  hdb_df = pd.read_csv('datasets/train.csv')


(150634, 77)
       id Tranc_YearMonth             town  flat_type block       street_name  \
0   88471         2016-05  KALLANG/WHAMPOA     4 ROOM    3B  UPP BOON KENG RD   
1  122598         2012-07           BISHAN     5 ROOM   153      BISHAN ST 13   
2  170897         2013-07      BUKIT BATOK  EXECUTIVE  289B    BT BATOK ST 25   
3   86070         2012-04           BISHAN     4 ROOM   232      BISHAN ST 22   
4  153632         2017-12           YISHUN     4 ROOM   876      YISHUN ST 81   

  storey_range  floor_area_sqm  flat_model  lease_commence_date  resale_price  \
0     10 TO 12            90.0     Model A                 2006      680000.0   
1     07 TO 09           130.0    Improved                 1987      665000.0   
2     13 TO 15           144.0   Apartment                 1997      838000.0   
3     01 TO 05           103.0     Model A                 1992      550000.0   
4     01 TO 03            83.0  Simplified                 1987      298000.0   

   Tranc_Year

---
## 3. Cleaning
---

### 3.1. Check for null values

In [91]:
pd.set_option('display.max_rows', None)  # to see all rows
missing_hdb_df = pd.DataFrame(hdb_df.isna().sum()).reset_index()
missing_hdb_df.columns = ['col', 'num_na']
missing_hdb_df['%na'] = missing_hdb_df['num_na']/len(hdb_df)*100
missing_hdb_df.style

,col,num_na,%na
0,id,0,0.000000
1,Tranc_YearMonth,0,0.000000
2,town,0,0.000000
3,flat_type,0,0.000000
4,block,0,0.000000
5,street_name,0,0.000000
6,storey_range,0,0.000000
7,floor_area_sqm,0,0.000000
8,flat_model,0,0.000000
9,lease_commence_date,0,0.000000


### 3.2. Change NaN values
* from 'X within Y distance' features to 0

In [92]:
hdb_df['Mall_Within_500m'] = [0 if math.isnan(x) else x for x in hdb_df['Mall_Within_500m']]
hdb_df['Mall_Within_1km'] = [0 if math.isnan(x) else x for x in hdb_df['Mall_Within_1km']]
hdb_df['Mall_Within_2km'] = [0 if math.isnan(x) else x for x in hdb_df['Mall_Within_2km']]
hdb_df['Hawker_Within_500m'] = [0 if math.isnan(x) else x for x in hdb_df['Hawker_Within_500m']]
hdb_df['Hawker_Within_1km'] = [0 if math.isnan(x) else x for x in hdb_df['Hawker_Within_1km']]
hdb_df['Hawker_Within_2km'] = [0 if math.isnan(x) else x for x in hdb_df['Hawker_Within_2km']]

* since kaggle needs predictions on all rows (i.e. cannot drop any rows), I am replacing all NaN 'Mall_Nearest_Distance' as 4km, beyond the maximum numeric value to represent that these NaN distances are far-out

In [93]:
print(hdb_df['Mall_Nearest_Distance'].max())
hdb_df['Mall_Nearest_Distance'] = [4000.0 if math.isnan(x) else x for x in hdb_df['Mall_Nearest_Distance']]

3496.40291


### 3.3. Change Y and N values
* to 1 and 0 respectively, so Y and N can become machine-readable

In [94]:
hdb_df['residential'] = [False if x == "N" else True for x in hdb_df['residential']]
hdb_df['commercial'] = [False if x == "N" else True for x in hdb_df['commercial']]
hdb_df['market_hawker'] = [False if x == "N" else True for x in hdb_df['market_hawker']]
hdb_df['multistorey_carpark'] = [False if x == "N" else True for x in hdb_df['multistorey_carpark']]
hdb_df['precinct_pavilion'] = [False if x == "N" else True for x in hdb_df['precinct_pavilion']]

### 3.4. Drop repetitive features 
* These values can be represented by other values (e.g. latitude and longitude values represented by location name)
* Also to avoid an expansion of features if get_dummies is used on these categorical data

In [95]:
# sanity check: rule against curse of dimensionality
# where N = 150,634, (N)**0.5 = 388 features
print(hdb_df.nunique().sort_values(ascending=False))

id                           150634
postal                        17483
address                        9157
Latitude                       9126
sec_sch_nearest_dist           9126
Hawker_Nearest_Distance        9126
mrt_nearest_distance           9126
pri_sch_nearest_distance       9126
bus_stop_nearest_distance      9126
Longitude                      9098
Mall_Nearest_Distance          8983
resale_price                   3295
block                          2514
bus_stop_latitude              2011
bus_stop_longitude             2011
bus_stop_name                  1657
street_name                     553
total_dwelling_units            320
3room_sold                      255
4room_sold                      200
pri_sch_name                    177
pri_sch_latitude                177
pri_sch_longitude               177
floor_area_sqft                 169
floor_area_sqm                  169
sec_sch_name                    134
sec_sch_latitude                134
sec_sch_longitude           

In [96]:
# drop 'repetitive' columns
hdb_df = hdb_df.drop(columns=['id', 'Tranc_YearMonth', 'lease_commence_date', 'flat_type', 'block', 'flat_model', 'storey_range', 'mid', 'lower', 'upper', 'postal', 'floor_area_sqft', 'block', 'street_name', 'address', 'planning_area', 'mrt_name', 'bus_stop_name', 'pri_sch_latitude', 'pri_sch_longitude', 'sec_sch_latitude', 'sec_sch_longitude', 'bus_stop_latitude', 'bus_stop_longitude', 'mrt_latitude', 'mrt_longitude', 'Longitude', 'Latitude'])

* Sanity checks

In [98]:
print(hdb_df.isnull().sum())  # results show none remaining

town                         0
floor_area_sqm               0
resale_price                 0
Tranc_Year                   0
Tranc_Month                  0
mid_storey                   0
full_flat_type               0
hdb_age                      0
max_floor_lvl                0
year_completed               0
residential                  0
commercial                   0
market_hawker                0
multistorey_carpark          0
precinct_pavilion            0
total_dwelling_units         0
1room_sold                   0
2room_sold                   0
3room_sold                   0
4room_sold                   0
5room_sold                   0
exec_sold                    0
multigen_sold                0
studio_apartment_sold        0
1room_rental                 0
2room_rental                 0
3room_rental                 0
other_room_rental            0
Mall_Nearest_Distance        0
Mall_Within_500m             0
Mall_Within_1km              0
Mall_Within_2km              0
Hawker_N

In [99]:
print(hdb_df.dtypes)  # looks good

town                          object
floor_area_sqm               float64
resale_price                 float64
Tranc_Year                    object
Tranc_Month                   object
mid_storey                     int64
full_flat_type                object
hdb_age                        int64
max_floor_lvl                  int64
year_completed                 int64
residential                     bool
commercial                      bool
market_hawker                   bool
multistorey_carpark             bool
precinct_pavilion               bool
total_dwelling_units           int64
1room_sold                     int64
2room_sold                     int64
3room_sold                     int64
4room_sold                     int64
5room_sold                     int64
exec_sold                      int64
multigen_sold                  int64
studio_apartment_sold          int64
1room_rental                   int64
2room_rental                   int64
3room_rental                   int64
o

---
## 4. Export data
---

In [100]:
newpath = 'output'
if not os.path.exists(newpath):
    os.makedirs(newpath)

hdb_df.to_csv('output/cleaned_baseline_hdb_train.csv', index=False)

In [101]:
# reset option to un-see rows and columns
pd.reset_option('display.max_rows')  # reset to un-see all rows
pd.reset_option('display.max_columns')  # reseting option to un-see all columns